In [ ]:
import numpy as np

from tensorflow import keras
from keras import layers
from keras.datasets import mnist

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

In [ ]:
# size of encoded representation
encoding_dim = 32 # 784 / 32 = 24.5 compression factor
# input image
input_img = keras.Input(shape=(784,))
# encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# lossy reconstruction of the input
decoded = layers.Dense(784, activation='sigmoid')(encoded)

In [ ]:
# maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)
autoencoder.summary()

In [ ]:
# maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)
encoder.summary()

In [ ]:
encoded_input = keras.Input(shape=(encoding_dim,))
# get the last layer of the autoencoder model to create decored
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))
decoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(
    x_train, 
    x_train,
    epochs=20,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test)
)

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
num_of_digits = 10  

plt.figure(figsize=(2 * num_of_digits, 4))

for i in range(num_of_digits):
    # Original
    ax = plt.subplot(2, num_of_digits, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Reconstruction
    ax = plt.subplot(2, num_of_digits, i + 1 + num_of_digits)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()